# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur agrégé au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>


# Chapitre 7 - Régression linéaire bayésienne




## Contexte : Poids des perchaudes du lac Laengelmavesi

Le fichier *fishweights.csv* contient les caractéristiques de 158 poissons péchés dans le lac Laengelmavesi en Finlande. Nous étudierons le poids (Weight en grammes) des poissons en fonction de variables explicatives suivantes :
- l'espèce de poisson (species) ;
- la longueur standard (StandardLength) ;
- la longeur non standard  (NonStandardLength) ;
- la longueur totale (TotalLength) ;
- la hauteur (Height) ;
- la largeur (Width).


          ___/////___                  _
         /           \    ___          |
       /\             \_ /  /          H
     <   )            __)  \           |
       \/_\\_________/   \__\          _

     |------- SL -------|
     |------- NSL ---------|
     |------- TL ------------|

In [1]:
using CSV 
using DataFrames
using Distributions
using Gadfly
using LinearAlgebra
import Random
using Statistics
import StatsBase

In [2]:
# using Cairo, Fontconfig

# Chargement des données

- Chargement du jeux de données
- Transformation du poids par la racine cubique pour linéariser la relation (voir le TD4)

In [ ]:
data = CSV.read("fishweights.csv", DataFrame)
data[!,:Weight] = data[:,:Weight] .^(1/3)
first(data,10)

## Sélection des perches

In [ ]:
filter!(row -> row.Species=="Perche", data)
first(data,10)

## Création du jeux de données d'entraînement et de validation

In [ ]:
Random.seed!(123)
train_id = sample(1:nrow(data), 44, replace=false, ordered=true)
valid_id = setdiff(1:nrow(data), train_id)

train = data[train_id, :]
valid = data[valid_id, :]

# 7.2 Régression bayésienne avec une loi *a priori* non informative

In [ ]:
y = train.Weight

n = length(y)

X = hcat(ones(n), train.StandardLength)

m = size(X,2)

β̂ = X\y
println("β̂ = ", β̂)

s² = 1/(n-m) * (y-X*β̂)'*(y-X*β̂)
println("s² = ", s²)

Σ = Symmetric(inv(X'X))

## 7.2.1 Lois conditionnelles complètes

In [ ]:
f₁(σ²::Real) = MvNormal(β̂,σ²*Σ)
f₂(β::Vector{<:Real}) = InverseGamma(n/2, .5(y-X*β)'*(y-X*β)) 

In [ ]:
niter = 10000

β = Array{Float64}(undef, 2, niter)
σ² = Array{Float64}(undef, niter)

β[:,1] = [0, .2]
σ²[1] = .25

for j in 2:niter
    β[:,j] = rand(f₁(σ²[j-1]))
    σ²[j] = rand(f₂(β[:,j]))
end

In [ ]:
fig = plot(y=β[1,:], Geom.line, Guide.xlabel("Itération"), Guide.ylabel("β₀"))
# draw(PDF("trace_beta0.pdf"), fig)

In [ ]:
# intervalle de crédibilité à 95% de β₀ estimé à partir de l'échantillon aléatoire généré
quantile(β[1,:], [.025, .975])

In [ ]:
fig = plot(y=β[2,:], Geom.line, Guide.xlabel("Itération"), Guide.ylabel("β₁"))
# draw(PDF("trace_beta1.pdf"), fig)

In [ ]:
fig = plot(y=σ², Geom.line, Guide.xlabel("Itération"), Guide.ylabel("σ²"))
# draw(PDF("trace_sigma2.pdf"), fig)

In [ ]:
# Retrait des itérations de chauffe
β = β[:, 101:end]
σ² = σ²[101:end];

## 7.2.2 Lois marginales

In [ ]:
f = LocationScale(β̂[1], sqrt(s²*Σ[1,1]), TDist(n-m))

fig = plot(Guide.xlabel("β₀"), Guide.ylabel("Densité"), Coord.cartesian(xmin=-.6, xmax=.5),
    layer(x->pdf(f, x), -.6, .5, Theme(default_color=colorant"red")),
    layer(x=β[1,:], Geom.histogram(density=true, bincount=30))
)
# draw(PDF("marg_beta0.pdf"), fig)

In [ ]:
f = LocationScale(β̂[2], sqrt(s²*Σ[2,2]), TDist(n-m))

fig = plot(Guide.xlabel("β₁"), Guide.ylabel("Densité"),
    layer(x->pdf(f, x), .24, .28, Theme(default_color=colorant"red")),
    layer(x=β[2,:], Geom.histogram(density=true, bincount=30))
)
# draw(PDF("marg_beta1.pdf"), fig)

In [ ]:
f = InverseGamma((n-m)/2,(n-m)/2*s²)

fig = plot(Guide.xlabel("σ²"), Guide.ylabel("Densité"),
    layer(x->pdf(f, x), 0, .2, Theme(default_color=colorant"red")),
    layer(x=σ², Geom.histogram(density=true, bincount=30))
)
# draw(PDF("marg_sigma2.pdf"), fig)

In [ ]:
fm = InverseGamma((n-m)/2,(n-m)/2*s²)
σ̂² = mode(fm)

fd = Normal.(X*β̂, sqrt(σ̂²))

ll = sum(logpdf.(fd,y))
bic = ll - (m+1)/2*log(n)

## 7.2.3 Prédiction

Prédiction du poids d'une perche de 20 cm

In [ ]:
x₀ = 20

y₀ = Float64[]

for j in eachindex(σ²)
    
    pd = Normal(β[1,j] + β[2,j]*x₀, σ²[j])
    
    push!(y₀, rand(pd))
    
end

In [ ]:
mean(y₀)

In [ ]:
quantile(y₀, [.025, .975])

In [ ]:
fig = plot(y=y₀, Geom.line, Guide.xlabel("Itération"), Guide.ylabel("ŷ₀"))
# draw(PDF("trace_y0.pdf"), fig)

In [ ]:
fig = plot(x=y₀, Geom.histogram(density=true, bincount=30),
    Guide.xlabel("y₀"), Guide.ylabel("Densité"))
# draw(PDF("marg_y0.pdf"), fig)

## Exemple 6 : Recherche du meilleur modèle unidimensionnel 

In [ ]:
df = DataFrame(Variable = Symbol[], R² = Float64[], BIC = Float64[], RMSE = Float64[], )

SST = y'y 

for v in Symbol.(names(train)[3:7])
    
    X = hcat(ones(n), train[:,v])
    
    m = size(X,2)
    β̂ = X\y
    
    e = (y-X*β̂)
    SSE = e'e
    s² = 1/(n-m-2) * SSE
    
    R² = round(1-SSE/SST, digits=4)

    fd = Normal.(X*β̂, sqrt(s²))
    bic = round(sum(logpdf.(fd,y)) - (m+1)/2*log(n), digits=2)
    
    X_valid = hcat(ones(nrow(valid)), valid[:,v])

    ŷ = X_valid * β̂

    rmse = round(StatsBase.rmsd(ŷ, valid.Weight), digits=4)
    
    push!(df, [v, R², bic, rmse])
end

sort!(df, :RMSE)

# 7.3 Regression Ridge

## Standardisation des variables

La standardisation s'effectue avec les fonctions `fit()` et `transform()` de la librairie *StatsBase.jl*. Les données peuvent être remises à leur échelle originale avec la fonction `reconstruct()`.

In [ ]:
# Estimation des paramètres de la standardisation
ty = StatsBase.fit(StatsBase.ZScoreTransform, train.Weight)

# Standardisation des variables
y = StatsBase.transform(ty, train.Weight)

# Estimation des paramètres de la standardisation
tx = StatsBase.fit(StatsBase.ZScoreTransform, Matrix{Float64}(train[:,3:7]), dims=1)

# Standardisation des variables
X = StatsBase.transform(tx, Matrix{Float64}(train[:,3:7]));

## Estimation des paramètres 

In [ ]:
n = length(y)
m = size(X,2)

λ = 3.72

β̂ᵣ = (X'X + λ*I)\X'y

s² = (y'y - β̂ᵣ'*(X'X + λ*I)*β̂ᵣ)/n

Σ = Symmetric(inv(X'X+λ*I))


println("β̂ᵣ = ", β̂ᵣ)
println("s² = ", s²)


## Échantillonnage de Gibbs 

In [ ]:
f₁(σ²::Real) = MvNormal(β̂ᵣ,σ²*Σ)
f₂(β::Vector{<:Real}) = InverseGamma((n+m)/2, .5(y-X*β)'*(y-X*β) + .5*λ*β'β) 

In [ ]:
niter = 1000

β = Array{Float64}(undef, m, niter)
σ² = Array{Float64}(undef, niter)

β[:,1] = zeros(m)
σ²[1] = .05

for j in 2:niter
    β[:,j] = rand(f₁(σ²[j-1]))
    σ²[j] = rand(f₂(β[:,j]))
end

In [ ]:
fig = plot(y=β[1,:], Geom.line, Guide.xlabel("Itération"), Guide.ylabel("β₁"))

In [ ]:
fig = plot(y=β[4,:], Geom.line, Guide.xlabel("Itération"), Guide.ylabel("β₄"))

In [ ]:
fig = plot(y=σ², Geom.line, Guide.xlabel("Itération"), Guide.ylabel("σ²"))

In [ ]:
# Retrait des itérations de la phase de chauffe

β = β[:, 101:1000]
σ² = σ²[101:1000];

## Lois marginales

In [ ]:
f = LocationScale(β̂ᵣ[1], sqrt(s²*Σ[1,1]), TDist(n))

fig = plot(Guide.xlabel("β₁"), Guide.ylabel("Densité"), Coord.cartesian(xmin=-.6, xmax=.75),
    layer(x->pdf(f, x), -.6, .75, Theme(default_color=colorant"red")),
    layer(x=β[1,:], Geom.histogram(density=true, bincount=30))
)
# draw(PDF("marg_beta0.pdf"), fig)

In [ ]:
f = LocationScale(β̂ᵣ[5], sqrt(s²*Σ[5,5]), TDist(n))

fig = plot(Guide.xlabel("β₅"), Guide.ylabel("Densité"), Coord.cartesian(xmin=0, xmax=.5),
    layer(x->pdf(f, x), 0, .5, Theme(default_color=colorant"red")),
    layer(x=β[5,:], Geom.histogram(density=true, bincount=30))
)

In [ ]:
f = InverseGamma(n/2,n*s²/2)

fig = plot(Guide.xlabel("σ²"), Guide.ylabel("Densité"),
    layer(x->pdf(f, x), .01, .05, Theme(default_color=colorant"red")),
    layer(x=σ², Geom.histogram(density=true, bincount=30))
)

## Trace des coefficients de régression en fonction de λ

In [ ]:
df = DataFrame(λ = Float64[], β₁ = Float64[], β₂ = Float64[], β₃ = Float64[], β₄ = Float64[], β₅ = Float64[])

for λ in 0:.01:6

    β̂ = (X'X + λ*I)\X'y
    
    push!(df, [λ, β̂...])
    
end

In [ ]:
trace = stack(df, Not(:λ))
rename!(trace, :variable => :Paramètre, :value => :Estimation)

set_default_plot_size(12cm, 8cm)
fig = plot(trace, x=:λ, y=:Estimation, color=:Paramètre, Geom.line,
    Coord.cartesian(xmin=0, xmax=6, ymin=-.5, ymax=.5))

# draw(PDF("trace_lambda.pdf"), fig)

## Spécification de λ

La valeur de λ choisie est celle qui minimise l'erreur de prédiction (rmse) sur l'échantillon de validation.

In [ ]:
# Tansformation des variables ex,plicatives de l'échantillon de validation
X_valid = StatsBase.transform(tx, Matrix{Float64}(valid[:,3:7]));

# Prédictions retransformées dans l'espaces originales
ŷ = StatsBase.reconstruct(ty, X_valid*β̂ᵣ)

In [ ]:
# RMSE pour la valeur de lambda

StatsBase.msd(ŷ, valid.Weight)

In [ ]:
df2 = DataFrame(λ = Float64[], mse = Float64[], rmse = Float64[])

# Tansformation des variables explicatives de l'échantillon de validation
X_valid = StatsBase.transform(tx, Matrix{Float64}(valid[:,3:7]));

for λ in 0:.01:6

    # Estimation des coefficients de régression ridge
    β̂ = (X'X + λ*I)\X'y
    
    # Prédictions retransformées dans l'espaces originales
    ŷ = StatsBase.reconstruct(ty, X_valid*β̂)
    
    # Calcul du mse
    mse = StatsBase.msd(ŷ, valid.Weight)
    
    # Calcul du rmse
    rmse = StatsBase.rmsd(ŷ, valid.Weight)
    
    push!(df2, [λ, mse, rmse])
    
end

In [ ]:
set_default_plot_size(21cm, 8cm)
hstack(
    plot(df2, x=:λ, y=:mse, Geom.line),
    plot(df2, x=:λ, y=:rmse, Geom.line),
    )

In [ ]:
set_default_plot_size(10.5cm, 8cm)
fig = plot(df2, x=:λ, y=:rmse, Geom.line)
# draw(PDF("rmse_lambda.pdf"), fig)

In [ ]:
sort!(df2, :rmse)
first(df2, 10)